# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
df_lng_lat = pd.read_csv("../output_data/lng_lat.csv")
df_lng_lat.head()
df_lng_lat['humidity'].max()

100

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
locations = df_lng_lat[['lat','lon']]
weights = df_lng_lat["humidity"]

In [9]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100, point_radius=1)


# # Add layer
fig.add_layer(heat_layer)

# # Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
df_lng_lat.head()

,Unnamed: 0,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date
0,0,Rikitea,-134.9692,-23.1203,77.86,76,16,19.86,PF,1620843479
1,1,Nishihara,127.7558,26.1842,78.03,93,75,11.14,JP,1620843604
2,2,Ponta do Sol,-17.1000,32.6667,68.00,69,100,3.80,PT,1620843557
3,3,Baturaja,104.1667,-4.1333,74.12,98,88,2.71,ID,1620843604
4,4,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605


In [11]:
#df['vaca_max_temp'] = df_lng_lat['max_temp'].apply(lambda x: True if [x <= 90 & x >= 70])
vaca_weather = df_lng_lat.loc[(df_lng_lat['max_temp']<= 90) & (df_lng_lat['max_temp']>= 65)]
vaca_weather = df_lng_lat.loc[(df_lng_lat['humidity']<= 30)]
vaca_weather.reset_index(inplace=True, drop = True)
vaca_weather.drop('Unnamed: 0', axis='columns', inplace=True)
vaca_weather.head()

/Users/jds/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date
0,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605
1,Banikoara,2.4386,11.2985,96.76,22,54,3.96,BJ,1620843616
2,Kumul,93.4500,42.8000,70.86,25,100,22.77,CN,1620843560
3,Tessalit,1.0114,20.1986,104.27,6,36,11.88,ML,1620843682
4,Jiuquan,98.5174,39.7432,59.27,23,77,2.24,CN,1620843685



### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
df_hotel = pd.DataFrame(vaca_weather)
df_hotel.head()

,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date
0,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605
1,Banikoara,2.4386,11.2985,96.76,22,54,3.96,BJ,1620843616
2,Kumul,93.4500,42.8000,70.86,25,100,22.77,CN,1620843560
3,Tessalit,1.0114,20.1986,104.27,6,36,11.88,ML,1620843682
4,Jiuquan,98.5174,39.7432,59.27,23,77,2.24,CN,1620843685


In [13]:
df_hotel['hotel_name'] = ""
df_hotel['hotel_address'] = ""
df_hotel['hotel_rating'] = ""
df_hotel.head()

,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date,hotel_name,hotel_address,hotel_rating
0,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605,,,
1,Banikoara,2.4386,11.2985,96.76,22,54,3.96,BJ,1620843616,,,
2,Kumul,93.4500,42.8000,70.86,25,100,22.77,CN,1620843560,,,
3,Tessalit,1.0114,20.1986,104.27,6,36,11.88,ML,1620843682,,,
4,Jiuquan,98.5174,39.7432,59.27,23,77,2.24,CN,1620843685,,,


In [21]:
#for debugging
# df_hotel = df_hotel[0:10]
# df_hotel

,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date,hotel_name,hotel_address,hotel_rating
0,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605,,,
1,Banikoara,2.4386,11.2985,96.76,22,54,3.96,BJ,1620843616,,,
2,Kumul,93.4500,42.8000,70.86,25,100,22.77,CN,1620843560,,,
3,Tessalit,1.0114,20.1986,104.27,6,36,11.88,ML,1620843682,,,
4,Jiuquan,98.5174,39.7432,59.27,23,77,2.24,CN,1620843685,,,
5,Ukiah,-123.2078,39.1502,82.74,24,1,3.44,US,1620843664,,,
6,Matagami,-77.6328,49.7502,54.19,28,1,9.71,CA,1620843760,,,
7,Benjamín Hill,-111.1000,30.1667,82.47,9,0,8.88,MX,1620843760,,,
8,Guadalupe y Calvo,-106.9667,26.1000,73.27,25,35,6.40,MX,1620843762,,,
9,Sohāgpur,78.2000,22.7000,91.63,21,25,9.84,IN,1620843770,,,


In [14]:
params = {
    "types": 'lodging',
    "radius": 5000,
    "key": g_key
}

for index, row in df_hotel.iterrows():
    lat = row['lat']
    lng = row['lon']
    
    params['location'] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    #print(name_address.url)
    try:
        df_hotel.loc[index, "hotel_name"] = name_address["results"][0]["name"]
        df_hotel.loc[index, "hotel_address"] = name_address["results"][0]["vicinity"]
        df_hotel.loc[index, "hotel_rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [15]:
df_hotel

,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date,hotel_name,hotel_address,hotel_rating
0,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605,Expeditions Ténére voyages,Bilma,4.8
1,Banikoara,2.4386,11.2985,96.76,22,54,3.96,BJ,1620843616,Hôtel de l'UCom CVPC,Banikoara,3.9
2,Kumul,93.4500,42.8000,70.86,25,100,22.77,CN,1620843560,Laoyutang Resort,"Yizhou, Hami",
3,Tessalit,1.0114,20.1986,104.27,6,36,11.88,ML,1620843682,,,
4,Jiuquan,98.5174,39.7432,59.27,23,77,2.24,CN,1620843685,Jiuquan Hotel,"33 Jie Fang Lu, Suzhou Qu, Jiuquan Shi",3.9
5,Ukiah,-123.2078,39.1502,82.74,24,1,3.44,US,1620843664,Hampton Inn Ukiah,"1160 Airport Park Boulevard, Ukiah",4.3
6,Matagami,-77.6328,49.7502,54.19,28,1,9.71,CA,1620843760,Hôtel Matagami,"99 Boulevard Matagami, Matagami",4
7,Benjamín Hill,-111.1000,30.1667,82.47,9,0,8.88,MX,1620843760,Hotel Benjamín Hill,"Sin nombre No. 12 LB, INFONAVIT, Benjamín Hill",
8,Guadalupe y Calvo,-106.9667,26.1000,73.27,25,35,6.40,MX,1620843762,Hotel Pizarro,"Corona, El Palmito, Guadalupe y Calvo",4
9,Sohāgpur,78.2000,22.7000,91.63,21,25,9.84,IN,1620843770,Hotel Vatika Inn- A unit of Hotel Ramayan,"near R.R.Jaiswal Petrol Pump, Sohagpur",3.8


In [16]:
df_hotel.to_csv("vacation_hotel.csv")
df_hotel = pd.read_csv('vacation_hotel.csv')

In [17]:
df_hotel

,Unnamed: 0,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date,hotel_name,hotel_address,hotel_rating
0,0,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605,Expeditions Ténére voyages,Bilma,4.8
1,1,Banikoara,2.4386,11.2985,96.76,22,54,3.96,BJ,1620843616,Hôtel de l'UCom CVPC,Banikoara,3.9
2,2,Kumul,93.4500,42.8000,70.86,25,100,22.77,CN,1620843560,Laoyutang Resort,"Yizhou, Hami",NaN
3,3,Tessalit,1.0114,20.1986,104.27,6,36,11.88,ML,1620843682,NaN,NaN,NaN
4,4,Jiuquan,98.5174,39.7432,59.27,23,77,2.24,CN,1620843685,Jiuquan Hotel,"33 Jie Fang Lu, Suzhou Qu, Jiuquan Shi",3.9
5,5,Ukiah,-123.2078,39.1502,82.74,24,1,3.44,US,1620843664,Hampton Inn Ukiah,"1160 Airport Park Boulevard, Ukiah",4.3
6,6,Matagami,-77.6328,49.7502,54.19,28,1,9.71,CA,1620843760,Hôtel Matagami,"99 Boulevard Matagami, Matagami",4.0
7,7,Benjamín Hill,-111.1000,30.1667,82.47,9,0,8.88,MX,1620843760,Hotel Benjamín Hill,"Sin nombre No. 12 LB, INFONAVIT, Benjamín Hill",NaN
8,8,Guadalupe y Calvo,-106.9667,26.1000,73.27,25,35,6.40,MX,1620843762,Hotel Pizarro,"Corona, El Palmito, Guadalupe y Calvo",4.0
9,9,Sohāgpur,78.2000,22.7000,91.63,21,25,9.84,IN,1620843770,Hotel Vatika Inn- A unit of Hotel Ramayan,"near R.R.Jaiswal Petrol Pump, Sohagpur",3.8


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_hotel.iterrows()]
locations1 = df_hotel[["lat", "lon"]]

In [19]:
# Plot Heatmap
marker_layer = gmaps.symbol_layer(locations=locations1, fill_color='blue', info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))